### Carlos Morán Y Carlos Tardón Grupo 9

# Práctica 1.B Problema 1

Para cada problema se debe proponer una representación adecuada que permita a un agente resolverlo con distintos
parámetros de entrada (tamaños, valores ,..), explicando las decisiones de representación, coste, heurística y descripción de los resultados de los distintos algoritmos.
Justificar cuál es el/los algoritmo/s de búsqueda utilizado para resolverlo, 
incluyendo datos de la complejidad de la resolución del problema.

Un grupo de 5 personas quiere cruzar un viejo y estrecho puente. Es una noche
cerrada y se necesita llevar una linterna para cruzar. El grupo solo dispone de
una linterna, a la que le quedan 5 minutos de batería.
<ul>
<li>Cada persona tarda en cruzar 10, 30, 60, 80 y 120 segundos,
respectivamente.</li>
<li>El puente solo resiste un máximo de 2 personas cruzando a la vez, y
cuando cruzan dos personas juntas, caminan a la velocidad del más lento.</li>
<li>No se puede lanzar la linterna de un extremo a otro del puente, así que
cada vez que crucen dos personas, alguien tiene que volver a cruzar hacia
atrás con la linterna a buscar a los compañeros que falten, y así hasta que
hayan cruzado todos.</li>
</ul>

### Representación Estados
Sea t_persona = [10, 30, 60, 80, 120] el array con el tiempo que cada persona tarda en cruzar, y t0 = 300 los segundos restantes de batería

Ideas, distintas formas:
<ol>
    <li> ((1, 0, 0, 1), l, t) donde tupla[0][i] indica orilla donde está la persona i, l=orilla de linterna(0,1) y t el tiempo que ha pasado desde que encendieron la linterna. 0<=t<=t0 </li>
    <li> ((i1,..,in), l, t), donde t indica el tiempo que ha pasado desde que encendieron la linterna. 0<=t<=t0, l=orilla de linterna(0,1), y la tupla tupla[0] indica los indices de las personas que están en la orilla 0. Supone un ahorro en espacio con respecto a la opción anterior, pero aumentamos el coste en tiempo, pues para calcular las acciones y nuevos estados es necesario recorrer la tupla entera. </li>
    <li> Si el array de personas es muy grande, y queremos ahorrar tiempo, convendría la siguiente representación: (set0, l, t), donde t indica el tiempo, l=orilla de linterna(0,1), y set0 el conjunto de personas(índices del array t_persona) en la orilla 0. Esta representación tiene un inconveniente, y es que para usar la librería aima es necesario que los estados sean hashables, pero el set no tiene definido hash en python (habría que definirlo, con los inconvenientes que conlleva)</li>
</ul>

De momento se usará la opción 1, pues en el caso que hay que resolver, la cantidad de personas no es muy grande, y la opcion 1 asegura que la tupla es hashable, y los set de python no tienen predefinida una funcion hash

### Representación Acciones
Ideas, distintas formas:
<ol>
    <li>Tupla (i1, i2) donde i1 e i2 son indices validos de personas, 0 <= i1 < len(t_persona) y, o bien i2=None ( si solo cruza uno) o bien 0 <= i2 < len(t_persona). Para no tener distintas tuplas que representen la misma acción, se impone i1 < i2</li>
   
</ul>

### Elección Coste, Algoritmo y Complejidad
Costes posibles:
<ol>
    <li>Tiempo t en cada estado, es decir, el tiempo que ha pasado desde que encendieron la linterna. En este caso se usaría busqueda de coste uniforme o astar (cuando tengamos una heurística)</li>
    <li>Número de veces que cruzan el puente. Este coste sería interesante si se añaden nuevas restricciones (por ejemplo, que el puente se rompe si pasas n veces, con batería infinita). En este caso parece que interesa usar busqueda en anchura. Sin embargo, si los tiempos son positivos(de hecho lo son), si minimizamos el coste anterior, minimizamos este también, pues al minimizar el tiempo de uso de la linterna, de la orilla 0 a la 1 pasarán 2 personas siempre que se pueda, y solo volverá una persona de la 1 a la 0 para devolver la linterna, lo que garantiza que se minimice el número de veces que se pasa el puente. Por tanto, para este caso también se puede usar astar con alguna heurística para el coste anterior.</li>
</ol>

### Estados objetivo
Los estados objetivos serán aquellos de la forma ((1, 1,..,1),l,t),  l=1 y 0<=t<=t0

### Posibles heurísticas
<ol>
    <li>h1. Podemos relajar las condiciones y asumir infinitas linternas en cada lado. De esta manera, nadie tiene que volver una vez cruzan el puente. Por tanto, una heurística admisible se obtiene considerando la solución voraz a este problema relajado, que consiste en emparejar a las dos personas más lentas, después a las siguientes dos personas más lentas, y así. Por ejemplo, si están [10, 30, 60, 80, 120] en la orilla 0, h1 = max(120,80) + max(60,30) + 10 = 190</li>
    <li>h2 Para mejorar h1, si la linterna está en la orilla 1 y no es final, se puede sumar a h1 el mínimo tiempo que costaría llevar la linterna a la orilla0, es decir, el tiempo de la persona más rápida en la orilla 1. Sigue siendo admisible, pues h1 lo era, y el tiempo que hemos sumado se corresponde con una acción que tendremos que realizar si o si en la solución real. </li>
   
</ul>

In [39]:
from search import *

In [40]:
class Linternas(Problem):

    def __init__(self, t_persona = [10, 30, 60, 80, 120], t0=300):
        t_persona.sort()
        self.t_persona = t_persona
        self.t0 = t0
        self.initial = ((0,)*len(t_persona), 0, 0)
        self.analizados  = 0

    def actions(self, state):
        tupla, l, t = state
        accs = []
        for i, e in enumerate(tupla):
            if e==l and self.t0 >= t + self.t_persona[i]:
                accs.append((i,None))
            if e == l:
                for j, f in enumerate(tupla[i+1:],i+1):
                    if f==l and self.t0 >= t + max(self.t_persona[i], self.t_persona[j]):
                        accs.append((i,j)) 
        return accs

    def result(self, state, action):
        tupla, l, t = state
        i1, i2 = action
        newt = t
        li = list(tupla)
        li[i1] = (li[i1]+1 )% 2
        if i2 == None:
            newt += self.t_persona[i1] 
        else:
            newt += max(self.t_persona[i1], self.t_persona[i2])
            li[i2] = (li[i2]+1 )% 2
            
        return (tuple(li), (l+1 )% 2, newt)


    def goal_test(self, state):
        self.analizados += 1
        return all(state[0]) and state[1] == 1 and 0<=state[2]<=self.t0

    def path_cost(self, c, state1, action, state2):
        i1, i2 = action
        if i2 == None:
            return c + self.t_persona[i1]
        return c  + max(self.t_persona[i1], self.t_persona[i2])
    
    def h1(self, node):
        tupla, _, _ = node.state
        indices_orilla0 = [i for i,e in enumerate(tupla) if e == 0]
        i = len(indices_orilla0)-1
        total = 0
        while i >= 0:
            total += self.t_persona[indices_orilla0[i]]
            i -= 2
        return total
    
    def h2(self, node):
        h1 = self.h1(node)
        tupla, l, t = node.state
        if h1 == 0 or l == 0:
            return h1
        # como l == 1 sabemos que hay alguien en orilla1
        for i,e in enumerate(tupla):
            if e == 1:
                return h1 + self.t_persona[i]

    def init_check(self): # Realiza checkeo inicial, para no tener que llamar a la busqueda en casos sin solución.
        if max(self.t_persona) > self.t0 or (max(self.t_persona) == self.t0 and len(self.t_persona) > 2):
            return False
        
        return True
    

In [41]:
def resuelve_linternas(problem, algoritmo,print_sol=True, h=None):
    if problem.init_check():
        if h: 
            res = algoritmo(problem,h)
        else:
            res = algoritmo(problem)
        if res is None:
            print("Instancia sin solución")
        else:
            sol = res.solution()
            if print_sol: #Para calcular tiempos, no printeamos la solución
                print("Solución: {0}".format(sol))
                print("Longitud de la solución: {0}".format(len(sol)))
                print("Nodos analizados: {0}".format(problem.analizados))
                if h:
                    print("Heurística: {0}".format(h.__name__))
                print("Algoritmo: {0}".format(algoritmo.__name__))
                print(f"Coste solucion: {res.path_cost}")
    else: 
        print("Instancia sin solución")


In [22]:
linternas = Linternas()
resuelve_linternas(linternas,breadth_first_graph_search,True)

Solución: [(0, 1), (0, None), (0, 2), (0, None), (3, 4), (1, None), (0, 1)]
Longitud de la solución: 7
Nodos analizados: 719
Algoritmo: breadth_first_graph_search
Coste solucion: 290


In [12]:
%%timeit
linternas = Linternas()
resuelve_linternas(linternas,breadth_first_graph_search,False)

74.3 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Solucion: [(0, 1), (0, None), (0, 2), (0, None), (3, 4), (1, None), (0, 1)] de coste 290. 

In [23]:
linternas = Linternas()
resuelve_linternas(linternas,uniform_cost_search,True)

Solución: [(0, 1), (0, None), (3, 4), (1, None), (0, 1), (0, None), (0, 2)]
Longitud de la solución: 7
Nodos analizados: 689
Algoritmo: uniform_cost_search
Coste solucion: 290


In [14]:
%%timeit
linternas = Linternas()
resuelve_linternas(linternas,uniform_cost_search, False)

295 ms ± 81.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Solucion: [(0, 1), (0, None), (3, 4), (1, None), (0, 1), (0, None), (0, 2)] de coste 290. En este caso, debido a la restriccion temporal(de 5 min), existen varias soluciones ( que son la misma salvo permutación del orden de las acciones) que minimizan a la vez los dos costes.

In [24]:
linternas = Linternas()
resuelve_linternas(linternas,astar_search,True, linternas.h1)

Solución: [(0, 2), (0, None), (0, 1), (0, None), (3, 4), (1, None), (0, 1)]
Longitud de la solución: 7
Nodos analizados: 172
Heurística: h1
Algoritmo: astar_search
Coste solucion: 290


In [19]:
%%timeit
linternas = Linternas()
resuelve_linternas(linternas,astar_search,False, linternas.h1)

80.5 ms ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
linternas = Linternas()
resuelve_linternas(linternas,astar_search, True, linternas.h2)

Solución: [(0, 2), (0, None), (0, 1), (0, None), (3, 4), (1, None), (0, 1)]
Longitud de la solución: 7
Nodos analizados: 115
Heurística: h2
Algoritmo: astar_search
Coste solucion: 290


In [26]:
%%timeit
linternas = Linternas()
resuelve_linternas(linternas,astar_search, False, linternas.h2)

68.9 ms ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


En este caso, la búsqueda en anchura tan solo tarda 74.3 ms, mientras que con busqueda de coste uniforme, tardamos una media de 295 ms en encontrar la solucion, y se analizan 689 nodos. Asimismo, astar con la heurística h1 tarda tan solo 80.5 ms, lo que supone una reducción del 72% con respecto a la búsqueda de coste uniforme, y se analizan 172 nodos. Por último, h2 con astar tarda 68.9 ms, y se analizan solo 115 nodos.

In [30]:
linternas2 = Linternas([1,2,5,10],17)
resuelve_linternas(linternas2,breadth_first_graph_search, True)

Solución: [(0, 1), (0, None), (2, 3), (1, None), (0, 1)]
Longitud de la solución: 5
Nodos analizados: 140
Algoritmo: breadth_first_graph_search
Coste solucion: 17


In [32]:
linternas2 = Linternas([1,2,5,10],17)
resuelve_linternas(linternas2,uniform_cost_search, True)

Solución: [(0, 1), (0, None), (2, 3), (1, None), (0, 1)]
Longitud de la solución: 5
Nodos analizados: 194
Algoritmo: uniform_cost_search
Coste solucion: 17


In [34]:
linternas2 = Linternas([1,2,5,10],17)
resuelve_linternas(linternas2,astar_search,True, linternas2.h1)

Solución: [(0, 1), (0, None), (2, 3), (1, None), (0, 1)]
Longitud de la solución: 5
Nodos analizados: 36
Heurística: h1
Algoritmo: astar_search
Coste solucion: 17


In [36]:
linternas2 = Linternas([1,2,5,10],17)
resuelve_linternas(linternas2,astar_search,True,linternas2.h2)

Solución: [(0, 1), (0, None), (2, 3), (1, None), (0, 1)]
Longitud de la solución: 5
Nodos analizados: 24
Heurística: h2
Algoritmo: astar_search
Coste solucion: 17


#### En las siguientes celdas, se ve un caso en el que interesa usar búsqueda astar con la heurística h2(que es una heurística para el coste 1)  para minimizar el número de veces que se pasa el puente, pues breadth_first_graph_search analiza 2966 nodos, y astar solo 68

In [37]:
linternas2 = Linternas([1,2,5,10,20],345)
resuelve_linternas(linternas2,breadth_first_graph_search, True)

Solución: [(0, 1), (0, None), (0, 2), (0, None), (0, 3), (0, None), (0, 4)]
Longitud de la solución: 7
Nodos analizados: 2966
Algoritmo: breadth_first_graph_search
Coste solucion: 40


In [38]:
linternas2 = Linternas([1,2,5,10,20],345)
resuelve_linternas(linternas2,astar_search,True,linternas2.h2)

Solución: [(0, 2), (0, None), (0, 1), (0, None), (3, 4), (1, None), (0, 1)]
Longitud de la solución: 7
Nodos analizados: 68
Heurística: h2
Algoritmo: astar_search
Coste solucion: 33


#### Por último, un caso sin solución

In [42]:
linternas3 = Linternas([1,2,5,10],16)
resuelve_linternas(linternas3,astar_search,True, linternas3.h1)

Instancia sin solución
